In [ ]:
import sys
import os
from pyemd import emd
import numpy as np
from gensim import models
from gensim.models import word2vec
import numpy.linalg as la
from gensim.models.word2vec import Word2Vec
import scipy.spatial as spt
from scipy.spatial import distance
from pyemd import emd
from sklearn.metrics import euclidean_distances
import itertools
import collections
from numpy import dot, zeros, dtype, array, sqrt,double,array,sqrt, sum as np_sum
import time

In [ ]:
word_vec = None
array = None
global words_1
global words_2
global words_1_set
global words_2_set
global File
removed_words = ['sdkls']

In [ ]:
W = np.memmap("data/embed.dat", dtype=np.double, mode="r", shape=(3000000, 300))
with open("data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())


In [ ]:
vocab_dict = {w: k for k, w in enumerate(vocab_list)}

In [ ]:
def FileOpen(file1,file2):  
    global woduplicates
    woduplicates = None
    with open(file1) as f:
        global words_1
        words_1 = f.read().split()
        words_1 = [token for token in words_1 if token in vocab_dict.keys()]

    with open(file2) as f:
        global words_2
        words_2 = f.read().split()
        words_2 = [token for token in words_2 if token in vocab_dict.keys()]
    
    woduplicates = set(words_1 + words_2)
    
    return woduplicates

In [ ]:
def centroidOfFile(words):
    fileWords = set(words)
    numberWords = len(fileWords)
    VecMatrixWords =np.zeros((numberWords, 300))
    centroid =np.zeros((1, 300))
    for j,t in enumerate(fileWords):
        VecMatrixWords[j] = (W[[vocab_dict[t]]])
    centroid =np.zeros((1, 300))
    for j in range(300):
        for i in range(numberWords):
            centroid[0][j] =  centroid[0][j]+(VecMatrixWords[i][j])
        centroid[0][j]=  centroid[0][j]/300
    return(centroid)

In [ ]:
def Dictionaty2Files(content, woduplicates):
    doc_len = len(content)
    d = []
    for i, t in enumerate(woduplicates):
        d.append(content.count(t) / float(doc_len))
    return d 

In [ ]:
def distance_matrix():
    vocab_len = len(woduplicates)
    distance_matrix = zeros((vocab_len, vocab_len), dtype=float)
    for i, t1 in enumerate(woduplicates):
        #print(t1)
        for j, t2 in enumerate(woduplicates):
            #print(t2)
            distance_matrix[i][j] = sqrt(np_sum((W[[vocab_dict[t1]]] - W[[vocab_dict[t2]]])**2))
    return distance_matrix

In [ ]:
def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    numberOfFiles = len(r)
    return r

In [ ]:
path = os.path.join(os.path.expanduser('.'),'data','dataWMD20NEWS','docs')
fnames = list_files(path)
numberOfFiles = len(os.listdir(path))

In [ ]:
start = time.time()
queryDocument = input("Please enter query document path: ")
k=30
#/home/vishwani/Downloads/LAB/FINALsubmissionSVEN/data/dataWMD20NEWS/testdata/querydoc
d_wcd = dict()
WMD_DICT = dict()
filename1 = queryDocument
fnames = list_files(path)
numberFiles = len(fnames)
for fileName2 in fnames:
    filename2 = fileName2
    global woduplicates
    woduplicates = FileOpen(filename1,filename2)
    centroid1 = centroidOfFile(words_1)
    centroid2 = centroidOfFile(words_2)
    d = sqrt(np_sum((centroid1 - centroid2)**2))
    d_wcd.update({(filename2, d)})
dict_wcd = (sorted(d_wcd.items(), key=lambda x:x[1]))
k_wcd = dict_wcd[0:k]
k_wcd_array =  np.asarray(k_wcd)
kfiles_sorted_wcd = k_wcd_array[:,0]

In [ ]:
for fileName2 in kfiles_sorted_wcd:
    filename2 = fileName2
    #print(filename2)
    global woduplicates
    woduplicates = FileOpen(filename1,filename2)
    if((len(words_1)!=0)and len(words_2)!=0):
        d1 = np.array(Dictionaty2Files(words_1, woduplicates), dtype=double)
        d2 = np.array(Dictionaty2Files(words_2, woduplicates), dtype=double)
        distance = distance_matrix()
        distanceWMD = emd(d2,d1,distance)
        WMD_DICT.update({(filename2,distanceWMD)})
    else:
        print("skipped")
        print(filename2)
sorted_K_WMD = (sorted(WMD_DICT.items(), key=lambda x:x[1]))
sorted_K_WMD_array = np.asarray(sorted_K_WMD[0:k])
print(sorted_K_WMD_array)
end = time.time
print((end-start)/60)

In [ ]:
end = time.time()
print((end-start)/60)